# import

In [2]:
import torch
import torch.nn as nn
from collections import Counter
import ast
from torch.utils.data import Dataset, DataLoader
import torch
from torch.utils.data import DataLoader, TensorDataset
import gc

device = torch.device('mps')

In [17]:
import numpy as np

In [1]:
import pandas as pd
user = pd.read_csv('./clean_df.csv', index_col=0)
problem = pd.read_excel('./data/problemList.xlsx')


In [3]:
import ast
user['solvedProblemList'] = user['solvedProblemList'].apply(ast.literal_eval)
user['triedNotsolvedList'] = user['triedNotsolvedList'].apply(ast.literal_eval)

# 필터링

In [9]:
tier_1_df = user[user['tier'] == 1]
unique_problems_tier_1 = set().union(*tier_1_df['solvedProblemList'])

In [13]:
unique_problems_per_tier = user.groupby('tier')['solvedProblemList'].agg(lambda x: set().union(*x))


In [15]:
unique_problem_counts = unique_problems_per_tier.apply(len)


In [16]:
unique_problem_counts

tier
1       392
2       907
3       832
4      1205
5      2198
6      5114
7      5346
8      5772
9      7275
10     7308
11     8363
12     8419
13     8915
14     9887
15    11264
16    11476
17    12363
18    10857
19    12989
20    12733
21    13857
22    14305
23    15148
24    14360
25    16567
26    15430
27     9252
28    11049
29     7119
30     6216
31    12924
Name: solvedProblemList, dtype: int64

In [18]:
problem

,problemId,title,acceptedUserCount,level,key,bojTagId
0,1000,A+B,272273,1,"['implementation', 'arithmetic', 'math']","[102, 121, 124]"
1,1001,A-B,232650,1,"['implementation', 'arithmetic', 'math']","[102, 121, 124]"
2,1002,터렛,35292,8,"['case_work', 'geometry', 'math']","[137, 100, 124]"
3,1003,피보나치 함수,49732,8,['dp'],[25]
4,1004,어린 왕자,14695,8,"['geometry', 'math']","[100, 124]"
...,...,...,...,...,...,...
29161,31060,Bovine Acrobatics,11,14,"['data_structures', 'greedy', 'priority_queue'...","[175, 33, 59, 74, 80]"
29162,31061,Cycle Correspondence,9,12,"['ad_hoc', 'greedy']","[109, 33]"
29163,31062,Target Practice,6,17,"['case_work', 'implementation', 'prefix_sum', ...","[137, 102, 139, 141]"
29164,31063,Candy Cane Feast,24,12,"['ad_hoc', 'bruteforcing', 'implementation', '...","[109, 125, 102, 141]"


In [21]:
# 문제 번호데이터프레임에 0인 애들 가져와서 드롭해야됨
filtered_problems_set = set(problem[problem['level'] == 0]['problemId'].to_list())

user['solvedProblemList'] = user['solvedProblemList'].apply(lambda x: [problem for problem in x if problem not in filtered_problems_set])
user['triedNotsolvedList'] = user['triedNotsolvedList'].apply(lambda x: [problem for problem in x if problem not in filtered_problems_set])

In [22]:
unique_problems_per_tier = user.groupby('tier')['solvedProblemList'].agg(lambda x: set().union(*x))
unique_problem_counts = unique_problems_per_tier.apply(len)
unique_problem_counts

tier
1       392
2       907
3       832
4      1205
5      2198
6      5114
7      5346
8      5772
9      7275
10     7308
11     8363
12     8419
13     8915
14     9887
15    11264
16    11476
17    12363
18    10857
19    12989
20    12733
21    13857
22    14305
23    15148
24    14360
25    16567
26    15430
27     9252
28    11049
29     7119
30     6216
31    12924
Name: solvedProblemList, dtype: int64

In [26]:
# Problem number to check
problem_to_check = 1223

# Check if the problem number is in any set of the Series
is_problem_present = any(problem_to_check in problems_set for problems_set in unique_problems_per_tier)

print(is_problem_present)

# 레벨 0인거 문제 번호 없음

False


# 각 티어별로 ease 문제를 필터링하기

In [4]:
# 문제 번호데이터프레임에 0인 애들 가져와서 드롭해야됨
filtered_problems_set = set(problem[problem['level'] == 0]['problemId'].to_list())

user['solvedProblemList'] = user['solvedProblemList'].apply(lambda x: [problem for problem in x if problem not in filtered_problems_set])
user['triedNotsolvedList'] = user['triedNotsolvedList'].apply(lambda x: [problem for problem in x if problem not in filtered_problems_set])

In [11]:
# 일단 티어 1인것 부터 체크

df = user[user['tier'].isin([1])]


In [12]:
# 모든 문제의 고유 목록 생성
all_problems = set()
for row in df.itertuples():
    all_problems.update(row.solvedProblemList)
    all_problems.update(row.triedNotsolvedList)
all_problems = sorted(all_problems)

# 이진 상호작용 행렬 생성
interaction_matrix = pd.DataFrame(0, index=df['id'], columns=all_problems)

for row in df.itertuples():
    interaction_matrix.loc[row.id, row.solvedProblemList] = 1

# 모델 학습 및 훈련

In [13]:
interaction_matrix

,1000,1001,1002,1003,10039,1004,1005,1008,1009,1010,...,9093,9095,9295,9316,9465,9498,9610,9613,9625,9654
id,,,,,,,,,,,,,,,,,,,,,
byooj0525,1,1,0,0,1,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
jiwoo5312,1,1,0,1,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
yubin00,1,1,0,0,1,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,1
suky4223,1,1,0,0,1,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,1
zzzbjs,1,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
h1358b,1,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
hw2402,1,1,0,1,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
imbrian618,1,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
junjunjunjun,1,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [14]:
# 데이터프레임을 PyTorch 텐서로 변환
train_tensor = torch.tensor(interaction_matrix.values).float().to(device)

In [15]:

gc.collect()
torch.mps.empty_cache()


class EASE(nn.Module):
    def __init__(self, num_items, reg_lambda=500):
        super(EASE, self).__init__()
        self.reg_lambda = reg_lambda
        self.num_items = num_items
        self.item_weights = nn.Parameter(torch.zeros(num_items, num_items))
        self.device = torch.device('mps')

    def forward(self, x):
        # x: 사용자-아이템 상호작용 행렬
        return torch.matmul(x, self.item_weights)
    def clear_memory(self):
        gc.collect()
        torch.cuda.empty_cache()
    
    def fit(self, data_loader):
        self.item_weights.data.zero_()  # 가중치 초기화

        for batch in data_loader:
            # 데이터 텐서 추출
            data_tensor = batch[0].to(self.device)

            # 배치에 대한 Gram Matrix 계산
            G_batch = torch.matmul(data_tensor.T, data_tensor) + self.reg_lambda * torch.eye(self.num_items, device=self.device)

            # Batch Gram Matrix의 역행렬 계산
            P_batch = torch.inverse(G_batch)

            # Batch Item Weights 계산
            B_batch = P_batch / -torch.diag(P_batch)
            B_batch.fill_diagonal_(0)

            # 각 배치의 결과를 누적
            self.item_weights.data += B_batch

        # 평균 가중치 계산
        self.item_weights.data /= len(data_loader)


# TensorDataset 및 DataLoader 생성
dataset = TensorDataset(train_tensor)
data_loader = DataLoader(dataset, batch_size=5000, shuffle=True)

model = EASE(num_items=train_tensor.shape[1], reg_lambda=100).to(device)

model.fit(data_loader)

In [18]:
solved_problems = df[df['id'] == 'byooj0525']['solvedProblemList'].to_list()[0]
# 문제 번호와 칼럼 인덱스의 매핑 생성
problem_to_index = {problem: idx for idx, problem in enumerate(interaction_matrix.columns)}
# 사용자가 풀었던 문제 번호를 인덱스로 변환
solved_problems_indices = [problem_to_index[problem] for problem in solved_problems if problem in problem_to_index]
# 전체 문제 수
num_problems = train_tensor.shape[1]

# 사용자 상호작용 텐서 생성
user_interaction_tensor = torch.zeros(num_problems)
user_interaction_tensor[solved_problems_indices] = 1
# 텐서를 모델이 있는 디바이스로 이동
user_interaction_tensor = user_interaction_tensor.to(device)
# 모델을 사용하여 추천 점수 계산
predicted_scores = model(user_interaction_tensor)

# 이미 풀었던 문제의 점수를 제외
predicted_scores[solved_problems_indices] = -np.inf

# 추천 점수 기반으로 추천 문제 선택 (예: 상위 N개 문제)
recommended_problems = torch.topk(predicted_scores, k=10).indices


In [19]:
recommended_problems

tensor([366,  42, 320, 360, 273, 161, 116, 230, 299, 384], device='mps:0')

In [20]:
# 칼럼 인덱스에서 문제 번호로의 역매핑 생성 -> 위에있는 값이 value라서 key로 찾을 수 없음 그래서 역매핑해야됨
index_to_problem = {idx: problem for problem, idx in problem_to_index.items()}
# 추천된 인덱스를 문제 번호로 변환
recommended_problem_numbers = [index_to_problem[idx.item()] for idx in recommended_problems]
recommended_problem_numbers =list(map(int, recommended_problem_numbers))
problem[problem['problemId'].isin(recommended_problem_numbers)]

,problemId,title,acceptedUserCount,level,key,bojTagId
65,1065,한수,67175,7,"['bruteforcing', 'math']","[125, 124]"
157,1157,단어 공부,83256,5,"['implementation', 'string']","[102, 158]"
316,1316,그룹 단어 체커,64234,6,"['implementation', 'string']","[102, 158]"
708,1712,손익분기점,55963,4,"['math', 'arithmetic']","[124, 121]"
1279,2292,벌집,64102,4,['math'],[124]
1529,2525,오븐 시계,81023,3,"['arithmetic', 'math']","[121, 124]"
1679,2675,문자열 반복,89111,4,"['implementation', 'string']","[102, 158]"
2047,3046,R2,29161,2,"['implementation', 'arithmetic', 'math']","[102, 121, 124]"
3309,4344,평균은 넘겠지,79850,5,"['arithmetic', 'math']","[121, 124]"


# 한번만 더 테스트

In [30]:
df = user[user['tier'].isin([15])]

In [31]:
# 문제 번호의 빈도를 계산하기 위한 Counter 객체 초기화
problem_counts = Counter()

# solvedProblemList 칼럼에서 문제 번호의 빈도를 계산
for problem_list in df['solvedProblemList']:
    problem_counts.update(problem_list)


# triedNotsolvedList 칼럼에서 문제 번호의 빈도를 계산
for problem_list in df['triedNotsolvedList']:
    problem_counts.update(problem_list)

In [32]:
# problem_counts에서 값이 5000 이상인 항목만 필터링
filtered_problems = {problem: count for problem, count in problem_counts.items() if count >= 1000}

# 필터링된 문제 번호 집합 생성
filtered_problems_set = set(filtered_problems.keys())

# solvedProblemList와 triedNotsolvedList에서 필터링된 문제 번호만 유지
df['solvedProblemList'] = df['solvedProblemList'].apply(lambda x: [problem for problem in x if problem in filtered_problems_set])
df['triedNotsolvedList'] = df['triedNotsolvedList'].apply(lambda x: [problem for problem in x if problem in filtered_problems_set])


/var/folders/d6/d15px6nn599bb66tcmrq23380000gn/T/ipykernel_9178/4253887450.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['solvedProblemList'] = df['solvedProblemList'].apply(lambda x: [problem for problem in x if problem in filtered_problems_set])
/var/folders/d6/d15px6nn599bb66tcmrq23380000gn/T/ipykernel_9178/4253887450.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['triedNotsolvedList'] = df['triedNotsolvedList'].apply(lambda x: [problem for problem in x if problem in filtered_problems_se

In [34]:
# 모든 문제의 고유 목록 생성
all_problems = set()
for row in df.itertuples():
    all_problems.update(row.solvedProblemList)
    all_problems.update(row.triedNotsolvedList)
all_problems = sorted(all_problems)

# 이진 상호작용 행렬 생성
interaction_matrix = pd.DataFrame(0, index=df['id'], columns=all_problems)

for row in df.itertuples():
    interaction_matrix.loc[row.id, row.solvedProblemList] = 1

In [35]:
interaction_matrix

,1000,1001,1002,10026,1003,10039,1004,1005,1008,1009,...,9375,9461,9465,9466,9498,9613,9653,9655,9663,9935
id,,,,,,,,,,,,,,,,,,,,,
jh5154,1,1,0,0,1,1,0,0,1,0,...,0,1,1,1,1,1,0,0,1,1
baefr,1,1,1,1,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,1,0
nacean,1,1,0,1,1,1,0,0,1,1,...,1,1,0,0,1,0,0,0,1,0
klxex,0,0,0,1,1,1,0,1,0,0,...,1,1,1,1,0,0,0,0,1,1
qortndud97,1,1,0,0,0,1,0,1,1,1,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
smmoon2000,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
dec0dedd,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
csm0626,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
# 데이터프레임을 PyTorch 텐서로 변환
train_tensor = torch.tensor(interaction_matrix.values).float().to(device)

In [37]:
# TensorDataset 및 DataLoader 생성
dataset = TensorDataset(train_tensor)

In [39]:
data_loader = DataLoader(dataset, batch_size=5000, shuffle=True)

model = EASE(num_items=train_tensor.shape[1], reg_lambda=100).to(device)

model.fit(data_loader)

In [40]:
df

,id,tier,rank,rightCnt,wrongCnt,time out,memory exceed,print exceed,runTime error,compile error,solvedProblemList,triedNotsolvedList
0,jh5154,15,10033,546,252,76,2,7,42,33,"[1000, 1001, 1003, 1008, 1012, 1018, 1021, 102...","[1916, 11279]"
1,baefr,15,10034,515,235,100,34,5,53,26,"[1000, 1001, 1002, 1008, 1010, 1011, 1012, 101...","[1062, 2343, 6588]"
2,nacean,15,10035,423,339,107,20,6,36,40,"[1000, 1001, 1003, 1008, 1009, 1010, 1012, 101...","[1038, 1937, 2407, 9935, 11650]"
3,klxex,15,10036,451,271,88,35,4,83,45,"[1003, 1005, 1010, 1012, 1018, 1021, 1038, 104...","[1253, 2133, 2473, 7662, 14002]"
4,qortndud97,15,10037,570,250,57,32,14,35,38,"[1000, 1001, 1005, 1008, 1009, 1010, 1012, 103...","[1865, 2609, 2839, 2884, 4195, 6064, 11657, 13..."
...,...,...,...,...,...,...,...,...,...,...,...,...
63966,smmoon2000,15,73757,82,2,17,718,106,390,45,"[1000, 1008, 1463, 1699, 2042, 2193, 2357, 249...","[2225, 2839, 9012]"
65820,dec0dedd,15,76096,79,18,232,93,0,30,7,"[1000, 1699]",[]
65821,csm0626,15,76099,79,12,233,95,79,21,17,"[1010, 1074, 1107, 1182, 1197, 1389, 1463, 147...","[1261, 1504, 1520, 2667, 9465, 11657, 11723]"
75555,taeyeon_ss,15,88580,67,1,3,193,0,7,96,[],[]


In [41]:
solved_problems = df[df['id'] == 'jh5154']['solvedProblemList'].to_list()[0]
# 문제 번호와 칼럼 인덱스의 매핑 생성
problem_to_index = {problem: idx for idx, problem in enumerate(interaction_matrix.columns)}
# 사용자가 풀었던 문제 번호를 인덱스로 변환
solved_problems_indices = [problem_to_index[problem] for problem in solved_problems if problem in problem_to_index]
# 전체 문제 수
num_problems = train_tensor.shape[1]

# 사용자 상호작용 텐서 생성
user_interaction_tensor = torch.zeros(num_problems)
user_interaction_tensor[solved_problems_indices] = 1
# 텐서를 모델이 있는 디바이스로 이동
user_interaction_tensor = user_interaction_tensor.to(device)
# 모델을 사용하여 추천 점수 계산
predicted_scores = model(user_interaction_tensor)

# 이미 풀었던 문제의 점수를 제외
predicted_scores[solved_problems_indices] = -np.inf

# 추천 점수 기반으로 추천 문제 선택 (예: 상위 N개 문제)
recommended_problems = torch.topk(predicted_scores, k=10).indices


In [42]:
# 칼럼 인덱스에서 문제 번호로의 역매핑 생성 -> 위에있는 값이 value라서 key로 찾을 수 없음 그래서 역매핑해야됨
index_to_problem = {idx: problem for problem, idx in problem_to_index.items()}
# 추천된 인덱스를 문제 번호로 변환
recommended_problem_numbers = [index_to_problem[idx.item()] for idx in recommended_problems]
recommended_problem_numbers =list(map(int, recommended_problem_numbers))
problem[problem['problemId'].isin(recommended_problem_numbers)]

,problemId,title,acceptedUserCount,level,key,bojTagId
149,1149,RGB거리,44529,10,['dp'],[25]
181,1181,단어 정렬,52580,6,"['sorting', 'string']","[97, 158]"
868,1874,스택 수열,39615,9,"['data_structures', 'stack']","[175, 71]"
909,1916,최소비용 구하기,18154,11,"['dijkstra', 'graphs', 'shortest_path']","[22, 7, 215]"
924,1932,정수 삼각형,38475,10,['dp'],[25]
1459,2455,지능형 기차,18316,3,"['arithmetic', 'implementation', 'math']","[121, 102, 124]"
8805,10026,적록색약,26424,11,"['bfs', 'dfs', 'graphs', 'graph_traversal']","[126, 127, 7, 11]"
10017,11279,최대 힙,26938,9,"['data_structures', 'priority_queue']","[175, 59]"
13099,14499,주사위 굴리기,15783,12,"['implementation', 'simulation']","[102, 141]"
